In [ ]:

import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp
from genpen.utils import Paper
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
import vpype
from skimage import io
from pathlib import Path

from sklearn.preprocessing import minmax_scale
from skimage import feature
from skimage import exposure

from skimage import filters
from skimage.color import rgb2gray
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.morphology import disk
from pyaxidraw import axidraw   # import module

from PIL import Image


import cv2

from genpen.flow.field import *
from genpen.flow.particle import *

import time
import fn
from genpen.axicam import AxiCam
%load_ext autoreload
%autoreload 2

In [ ]:

def scaled_copy_shade(poly, scale_factors=None, n_scales=8):
    if scale_factors is None:
        scale_factors = np.linspace(0., 1., n_scales)
    scaled_boundaries = [sa.scale(poly.boundary, xfact=sf, yfact=sf) for sf in scale_factors]
    return MultiLineString(scaled_boundaries)


def hollow_and_shade(poly, scale_factors=None, n_scales=8):
    shades = scaled_copy_shade(poly, scale_factors=scale_factors, n_scales=n_scales)
    shade_mls = gp.merge_LineStrings(shades)
    return MultiLineString([poly.boundary] + list(shade_mls))


def hollow_and_morse_shade(poly, scale_factors=None, n_scales=8, buffer_factor=0.01):
    shades = scaled_copy_shade(poly, scale_factors=scale_factors, n_scales=n_scales)
    shade_mls = gp.merge_LineStrings([morsify(s, buffer_factor=buffer_factor) for s in shades])
    return MultiLineString([poly.boundary] + list(shade_mls))


def morsify(ls, buffer_factor=0.01):
    return ls.buffer(buffer_factor).buffer(-buffer_factor).boundary

In [ ]:

# filename = 'vp_test29.svg'
paper_size:str = '6x6 inches'
border:float=10  # mm

savedir='/home/naka/art/plotter_svgs'

In [ ]:
# make page
paper = Paper(paper_size)
drawbox = paper.get_drawbox(border)


In [ ]:

xs = np.linspace(0, 6, 4) 
ys = np.linspace(0, 6, 4)

points = []
for x in xs:
    for y in ys:
        points.append(Point(x,y))

In [ ]:
circles = [gp.RegPolygon(p, radius=0.4, n_corners=6, rotation=np.random.uniform(360)).poly.buffer(1.2) for p in points]

In [ ]:

sgen = lambda: np.random.randint(low=8,high=40)
bgen = lambda: np.random.uniform(0.01, 0.05)
bcs = gp.merge_LineStrings([hollow_and_morse_shade(c, n_scales=sgen(), buffer_factor=bgen()) for c in circles])

In [ ]:
bcs = MultiLineString([l for l in bcs if l.length>0.01])
bcs = gp.make_like(bcs, drawbox)

In [ ]:
bcs = bcs.buffer(0.6, cap_style=2, join_style=2).boundary

In [ ]:
bcs = MultiLineString([l for l in bcs if l.length>1])

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.stroke(1)

sk.geometry(bcs)
    
sk.display(color_mode='none')

In [ ]:
drawbox.bounds

In [ ]:
filename = 'mazes.svg'

In [ ]:
sk.vpype('linemerge linesimplify linesort')

In [ ]:
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)

In [ ]:
bcs = gp.make_like(bcs, drawbox)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

counter = 0
for ls in bcs:
    counter += 1
    sk.stroke(counter)
    sk.geometry(ls)
    

sk.display(color_mode='layer')

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savedir='/home/naka/art/plotter_svgs'

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

In [ ]:
ac = AxiCam()

In [ ]:
ac.init_cam()

In [ ]:
ac.cam.video_preview()

In [ ]:
ac.cam.close()

In [ ]:
ac.ad.options.speed_penup = 80

In [ ]:
ac.ad.options.accel = 70
ac.ad.options.pen_raise_rate = 85

In [ ]:
ac.ad.options.speed_pendown = 60
ac.ad.update()

In [ ]:
ac.toggle_pen()

In [ ]:
ac.plot_layers(wait_times=0.3, start_layer=0)

In [ ]:
ac.cam.get_video(wait_time=3)